# PySpark Tutorial

## PySpark Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tutorial").getOrCreate()

In [3]:
data = [
    ("Agus Richard", 26, "Richard", "Male"),
    ("Damara Astiningtyas", 23, "Ara", "Female")
]
column = ["Name", "Age", "Nickname", "Gender"]

df = spark.createDataFrame(data, column)
df.show()

+-------------------+---+--------+------+
|               Name|Age|Nickname|Gender|
+-------------------+---+--------+------+
|       Agus Richard| 26| Richard|  Male|
|Damara Astiningtyas| 23|     Ara|Female|
+-------------------+---+--------+------+


In [4]:
df.createOrReplaceTempView("sample_table")
df2 = spark.sql("SELECT Name FROM sample_table")
df2.show()

+-------------------+
|               Name|
+-------------------+
|       Agus Richard|
|Damara Astiningtyas|
+-------------------+


In [5]:
spark.table("sample_table").write.saveAsTable("sample_hive_table")

SparkRuntimeException: [LOCATION_ALREADY_EXISTS] Cannot name the managed table as `spark_catalog`.`default`.`sample_hive_table`, as its associated location 'file:/Users/agusrichard/Documents/personal/workbook/data-engineering-workbook/pyspark-tutorial/playground/spark-warehouse/sample_hive_table' already exists. Please pick a different table name, or remove the existing location first.

In [ ]:
df3 = spark.sql("SELECT * FROM sample_hive_table")
df3.show()

In [ ]:
dbs = spark.catalog.listDatabases()
print(dbs)

In [ ]:
tables = spark.catalog.listTables()
for table in tables:
    print(table.name)

## PySpark RDD

In [ ]:
spark = SparkSession.builder.appName("pyspark-rdd").getOrCreate()

In [ ]:
data = list(range(100))
print(data)

In [ ]:
rdd = spark.sparkContext.parallelize(data, 10)

In [ ]:
rdd.getNumPartitions()

In [ ]:
rdd = spark.sparkContext.textFile("./test.txt")

In [ ]:
rdd2 = rdd.flatMap(lambda line: line.split(" "))
rdd3 = rdd2.map(lambda x: (x, 1))
rdd4 = rdd3.map(lambda x: (x[1], x[0])).sortByKey()
print(rdd4.collect())

In [ ]:
rdd4.saveAsTextFile("result_test")
# somethihgn

## Repartition and Coalesce

In [9]:
spark = SparkSession.builder.appName("repartition-coalesce").master("local[4]").getOrCreate()

24/01/30 16:44:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [10]:
rdd = spark.sparkContext.parallelize(list(range(1000)))
print("N partitions: ", rdd.getNumPartitions())

N partitions:  8


In [11]:
rdd2 = rdd.repartition(4)
print("N partitions: ", rdd2.getNumPartitions())

N partitions:  4


In [13]:
rdd2.saveAsTextFile("./tmp/repartition")

## Broadcast and Accumulator Variables

In [15]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

rdd = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)

24/01/30 17:04:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


In [16]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("accumulator").getOrCreate()

accum=spark.sparkContext.accumulator(0)
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x
rdd.foreach(countFun)
print(accuSum.value)

accumCount=spark.sparkContext.accumulator(0)
rdd2=spark.sparkContext.parallelize([1,2,3,4,5])
rdd2.foreach(lambda x:accumCount.add(1))
print(accumCount.value)

24/01/30 17:04:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


15


15
5


## PySpark Dataframe

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)

df = rdd.toDF()
df.printSchema()
df.show(truncate=False)

deptColumns = ["dept_name","dept_id"]
df2 = rdd.toDF(deptColumns)
df2.printSchema()
df2.show(truncate=False)

deptDF = spark.createDataFrame(rdd, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

from pyspark.sql.types import StructType,StructField, StringType
deptSchema = StructType([       
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True)
])

deptDF1 = spark.createDataFrame(rdd, schema = deptSchema)
deptDF1.printSchema()
deptDF1.show(truncate=False)

24/02/05 20:58:34 WARN Utils: Your hostname, agusrichard.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
24/02/05 20:58:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/05 20:58:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)
+---------+---+
|_1       |_2 |
+---------+---+
|Finance  |10 |
|Marketing|20 |
|Sales    |30 |
|IT       |40 |
+---------+---+
root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)
+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+
root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)
+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: string (nullable = true)
+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+


In [2]:
from pyspark.sql.types import StructType, StructField, StringType,IntegerType
dataStruct = [(("James","","Smith"),"36636","M","3000"), \
      (("Michael","Rose",""),"40288","M","4000"), \
      (("Robert","","Williams"),"42114","M","4000"), \
      (("Maria","Anne","Jones"),"39192","F","4000"), \
      (("Jen","Mary","Brown"),"","F","-1") \
]

schemaStruct = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
          StructField('dob', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', StringType(), True)
         ])
df = spark.createDataFrame(data=dataStruct, schema = schemaStruct)
df.printSchema()

pandasDF2 = df.toPandas()
print(pandasDF2)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
                   name    dob gender salary
0      (James, , Smith)  36636      M   3000
1     (Michael, Rose, )  40288      M   4000
2  (Robert, , Williams)  42114      M   4000
3  (Maria, Anne, Jones)  39192      F   4000
4    (Jen, Mary, Brown)             F     -1
